# 크롤링 실습

In [2]:
from newspaper import Article

In [3]:
url = 'https://view.asiae.co.kr/article/2023032217473512545?utm_source=newsstand.naver.com&utm_medium=referral&utm_campaign=top6' 

In [7]:
article=Article(url, language='ko')
#아티클 정의=아티클(초기화, 언어 한국어로 지정)

In [8]:
article.download()
# 아티클 다운로드

In [9]:
article.parse()

In [ ]:
print(article.text)
# 저장한 본문 프린트

In [ ]:
article.title
# 타이틀 가져오기

In [ ]:
article.publish_date
# 기사 날짜

In [20]:
article.authors
# 다른 부분에 기자정보있어서 공란으로 나옴

[]

In [ ]:
# 기사 html 불러오기
article.html

In [ ]:
# 기자이름 크롤링
import re
re.findall('<div class="reporter">', article.html)

In [ ]:
article.html.split('<div class="reporter">')[1].split('<strong>')[1].split('</strong>')[0]

In [ ]:
# 기사 헤드라이너 크롤링
import re
re.findall('<div class="article_head">', article.html)

In [ ]:
article.html.split('<div class="article_head">')[1].split('<h4>')[1].split('</h4>')[0]
# <div class="article_head"> 태그와 <h4> 태그 사이에 있는 텍스트를 추출

In [ ]:
# 기사 로고 크롤링
import re
re.findall('<div class="logo">', article.html)
# article.html에 저장된 HTML 코드에서 <div class="logo"> 태그와 일치하는 모든 부분을 찾아 리스트로 반환

In [ ]:
article.html.split('<strong>')[1].split('</strong>')[0]
# 이때 분할된 결과에서 인덱스 1에 해당하는 부분을 선택합니다. 즉, <strong> 태그 다음에 나오는 부분을 선택합니다.
# 선택된 부분을 </strong> 태그를 기준으로 분할합니다.
# 이때 분할된 결과에서 인덱스 0에 해당하는 부분을 선택합니다. 즉, </strong> 태그 이전의 텍스트 부분을 선택합니다.

In [ ]:
# onclick 속성 추출
onclick_attribute = re.search(r'onclick="(.*?)"',article.html).group(1)

# 이벤트 카테고리 추출
event_category = re.search(r"'event_category' : '(.*?)'", onclick_attribute).group(1)

# 이벤트 라벨 추출
event_label = re.search(r"'event_label' : '(.*?)'", onclick_attribute).group(1)

print("이벤트 카테고리:", event_category)
print("이벤트 라벨:", event_label)

# 알라딘 홈페이지 크롤링

In [60]:
#import package
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [67]:
#알라딘 주간 베스트셀러 주소
page=1 # 원하는 페이지 번호
for i in range (url = f'https://aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page='+ i + &cnt=1000&SortOrder=1)

req = requests.get(url) # 서버에 접속 정보 보내주기. 주소창에 주소넣고 엔터치는 거랑 같은 작업.
content = req.content # 변수 만들기: content = req.내용물에 대한 정보
soup = BeautifulSoup(content, 'html.parser') # soup로 저장 = 초기화, html형태로 파싱

In [ ]:
# 알라딘 주간 베스트셀러 html 불러오기
soup

#### 베스트셀러 도서명 크롤링

In [ ]:
# 도서명
title=[soup.select('a.bo3')[i].text for i in range(0, 50)]
title

In [ ]:
# 작가
soup.select('div.ss_book_list')[0].select('li')[2].select('a')[0].text
#soup.select('추출하려는 값에 가장 가까운 클래스 이름')[책 순서. 0은 첫번째책].select[li('추출하려는 값에 가장 가까운 태그')[태그 순서].selext('추출하려는 값에 가장 가까운 태그')[해당 태그의 순서]

In [ ]:
# 할인가
soup.select('div.ss_book_list')[0].select('li')[3].select('b')[0].text

In [185]:
author=[]
for i in range(0,100,2):
    try:
        author.append(soup.select('div.ss_book_list')[i].select('li')[2].select('a')[0].text)
    except:
        author.append(soup.select('div.ss_book_list')[i].select('li')[1].select('a')[0].text)
# 짝수 만 봄
#    우선 해봄
#    안되면 다음에 해봄
# 이름.append --> list에 원소 추가 한 번에 하나씩 이름에 추가

In [ ]:
#할인가 - 반복하여 추출

for i in range(0,100,2):
    try:
        print(soup.select('div.ss_book_list')[i].select('li')[3].select('b')[0].text)
    except:
        print(i)

In [199]:
# 가격 - 반복하여 추출
price=[]
for i in range(0,100,2):
    try:
        price.append(int(soup.select('div.ss_book_list')[i].select('ul')[0].select('li')[3].select('span')[0].text.replace(',',"")))
    except:
        price.append(int(soup.select('div.ss_book_list')[i].select('ul')[0].select('li')[2].select('span')[0].text.replace(',',"")))

In [ ]:
# 출간연도
import re
re.findall('\d\d\d\d년', soup.select('div.ss_book_list')[0].select('ul')[0].select('li')[2].text)[0]

In [ ]:
re.findall('\d\d\d\d년', soup.select('div.ss_book_list')[0].text)[0]

In [201]:
year=[re.findall('\d\d\d\d년', soup.select('div.ss_book_list')[i].select('ul')[0].text)[0] for i in range(0,100,2)]

In [203]:
import pandas as pd
result=pd.DataFrame([title, author, price, year], index=['제목', '저자', '정가', '출간연도']).T

In [ ]:
# 결과보기
result

In [198]:
# 엑셀파일로 결과 저장
result.to_excel('res.xlsx', index=False)